## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_0 = pd.read_csv("../../바이너리_최종데이터_1004/gcc6/o0/o0_bincore6.csv", index_col=0)
print(bin6_0.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_0.reset_index(inplace=True, drop=True)

print('shape -', bin6_0.shape)
print('reset_index 완료')
print('input data shape')
bin6_0.head()

(30092946, 2)
shape - (30092946, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_0['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_0['label'].value_counts())

256
0    30042688
1       50258
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_0[bin6_0['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 16 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_0), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_0', len(ls_idx_bin))

# loc 로 수정필요
bin6_0_Ngram = bin6_0.loc[ls_idx_bin,:].copy()

804128
나머지 0
최종 길이 804128
bin6_0 804128


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_0_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_0.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_0)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_0[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

50258.0
0 29371349
1000 16960659
2000 14387330
3000 16441457
4000 18595709
5000 17460781
6000 4762414
7000 7003024
8000 24449591
9000 7861935
10000 11156373
11000 21629183
12000 28547265
13000 26903801
14000 15663539
15000 19833548
16000 17269803
17000 5446724
18000 28313580
19000 17375310
20000 14519415
21000 23592409
22000 18020431
23000 21233792
24000 15023365
25000 25545981
26000 18464465
27000 12035482
28000 6257685
29000 26892054
30000 28194752
31000 15681970
32000 26150312
33000 54500
34000 13265371
35000 24167953
36000 4040979
37000 1878064
38000 24237954
39000 22658415
40000 26371308
41000 5678589
42000 22523871
43000 5135834
44000 9150785
45000 10250050
46000 17512458
47000 27108502
48000 13038070
49000 15564983
50000 29923311
완료
50258


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_0['label'].value_counts()[1])

50258
50258


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_0_Ngram])
final.shape

(1608256, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_0_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_0_onehot_Ngram = pd.concat([final['label'], bc6_0_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_0_onehot_Ngram.shape)

원핫인코딩완료
(1608256, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc6_0 = bc6_0_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_0 = bc6_0_onehot_Ngram['label'].to_numpy()
print(x_bc6_0.shape, x_bc6_0.shape)

x_bc6_0 = x_bc6_0.reshape(-1, right_idx, x_bc6_0.shape[1])
y_bc6_0 = y_bc6_0.reshape(-1, right_idx, 1)

print(x_bc6_0.shape, y_bc6_0.shape)

(1608256, 256) (1608256, 256)
(100516, 16, 256) (100516, 16, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_0.shape[0])

x_bc6_0 = x_bc6_0[p]
y_bc6_0 = y_bc6_0[p]

print(x_bc6_0.shape, y_bc6_0.shape)

(100516, 16, 256) (100516, 16, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(24, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc6_0, y_bc6_0):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc6_0[train],
               y_bc6_0[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc6_0[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc6_0[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 21s 230us/sample - loss: 0.0106 - accuracy: 0.9978
Epoch 2/10
90464/90464 [==============================] - 18s 202us/sample - loss: 3.8345e-04 - accuracy: 0.9999
Epoch 3/10
90464/90464 [==============================] - 19s 209us/sample - loss: 3.0108e-04 - accuracy: 0.9999
Ep

accuracy_score 0.9999005173099881
recall_score 0.9982527664531159
precision_score 0.9986405127209167
f1_score 0.9984466019417476
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 27s 296us/sample - loss: 2.5320e-04 - accuracy: 0.9999
Epoch 2/10
90464/90464 [==============================] - 23s 257us/sample - loss: 2.3527e-04

accuracy_score 0.9999689116593713
recall_score 0.9996222138269739
precision_score 0.9994334277620397
f1_score 0.9995278118802531
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 29s 318us/sample - loss: 2.2749e-04 - accuracy: 1.0000
Epoch 2/10
90464/90464 [==============================] - 30s 333us/sample - loss: 2.1031e-04

accuracy_score 0.9999440409868683
recall_score 0.9990188383045526
precision_score 0.9992149165848871
f1_score 0.9991168678245511
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 43s 477us/sample - loss: 2.0536e-04 - accuracy: 1.0000
Epoch 2/10
90464/90464 [==============================] - 38s 425us/sample - loss: 1.8210e-04

accuracy_score 0.999975129327497
recall_score 0.9998061639852684
precision_score 0.9994187173028483
f1_score 0.9996124031007751
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90464 samples
Epoch 1/10
90464/90464 [==============================] - 46s 509us/sample - loss: 1.5679e-04 - accuracy: 1.0000
Epoch 2/10
90464/90464 [==============================] - 39s 429us/sample - loss: 1.5283e-04 

...

accuracy_score 0.999975129327497
recall_score 0.9998058629392351
precision_score 0.9994178148651272
f1_score 0.9996118012422361
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90465 samples
Epoch 1/10
90465/90465 [==============================] - 61s 669us/sample - loss: 9.0003e-05 - accuracy: 1.0000
Epoch 2/10
90465/90465 [==============================] - 49s 537us/sample - loss: 9.2174e-05 

accuracy_score 0.9999875634265247
recall_score 1.0
precision_score 0.9996130030959752
f1_score 0.9998064640990905
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90465 samples
Epoch 1/10
90465/90465 [==============================] - 56s 619us/sample - loss: 9.1791e-05 - accuracy: 1.0000
Epoch 2/10
90465/90465 [==============================] - 48s 532us/sample - loss: 7.2732e-05 - accuracy: 1.

accuracy_score 0.9999813451397871
recall_score 1.0
precision_score 0.9994209612044007
f1_score 0.9997103967564437
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90465 samples
Epoch 1/10
90465/90465 [==============================] - 57s 625us/sample - loss: 7.8627e-05 - accuracy: 1.0000
Epoch 2/10
90465/90465 [==============================] - 50s 549us/sample - loss: 6.7937e-05 - accuracy: 1.

accuracy_score 0.9999875634265247
recall_score 1.0
precision_score 0.9996172248803827
f1_score 0.9998085758039816
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 90465 samples
Epoch 1/10
90465/90465 [==============================] - 62s 687us/sample - loss: 6.3641e-05 - accuracy: 1.0000
Epoch 2/10
90465/90465 [==============================] - 52s 579us/sample - loss: 4.7415e-05 - accuracy: 1.

accuracy_score 0.9999813451397871
recall_score 1.0
precision_score 0.9994161152199299
f1_score 0.9997079723547162

K-fold cross validation Accuracy: [0.9999005173099881, 0.9999689116593713, 0.9999440409868683, 0.999975129327497, 0.999950258654994, 0.999975129327497, 0.9999875634265247, 0.9999813451397871, 0.9999875634265247, 0.9999813451397871]

K-fold cross validation Recall: [0.9982527664531159, 0.9996222138269739, 0.9990188383045526, 0.9998061639852684, 0.999804228660924, 0.9998058629392351, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9986405127209167, 0.9994334277620397, 0.9992149165848871, 0.9994187173028483, 0.9986312084473993, 0.9994178148651272, 0.9996130030959752, 0.9994209612044007, 0.9996172248803827, 0.9994161152199299]

K-fold cross validation F1-Score: [0.9984466019417476, 0.9995278118802531, 0.9991168678245511, 0.9996124031007751, 0.9992173742907454, 0.9996118012422361, 0.9998064640990905, 0.9997103967564437, 0.9998085758039816, 0.9997079723547162]


## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999651804398839
10-Fold Cross_validation. Recall : 0.9996310074170071
10-Fold Cross_validation. Precision : 0.9992823902083907
10-Fold Cross_validation. F1-Score : 0.9994566269294541


In [13]:
model1.save('gcc6_bin_core_s16_h24_o0.h5')
print('save 완료')

save 완료
